In [71]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


First, prep the candidates:

In [1]:
from snorkel.parser import HTMLParser
html_parser = HTMLParser(path='data/hardware_docs/')

In [2]:
from snorkel.parser import TableParser
table_parser = TableParser()

In [40]:
from snorkel.parser import Corpus
%time corpus = Corpus(html_parser, table_parser, max_docs=1000)

Parsing documents...
Parsing contexts...
Parsed 98 documents and 630 contexts
CPU times: user 1min 6s, sys: 3.31 s, total: 1min 9s
Wall time: 1min 44s


In [86]:
from snorkel.candidates import TableNgrams
from snorkel.matchers import NumberMatcher, RangeMatcher

table_ngrams = TableNgrams(n_max=2)
number_matcher = RangeMatcher(low=-80,high=-40)

In [87]:
from snorkel.candidates import Candidates
%time candidates = Candidates(table_ngrams, number_matcher, corpus.get_contexts())

Extracting candidates...
CPU times: user 4.6 s, sys: 64 ms, total: 4.66 s
Wall time: 4.63 s


Next, prep the gold data:

In [102]:
import csv
with open('data/gold_stg_temp_min.csv', 'rb') as csvfile:
    gold_reader = csv.reader(csvfile)
    gold = []
    for row in gold_reader:
        (doc, part, temp, label) = row
        if label=='stg_temp_min':
            gold.append((doc,temp))
validation = 0.25
train = 1 - validation
gold = list(set(gold))
gold_validation = gold[int(len(gold)*train):]
gold_train = gold[:int(len(gold)*(train))]
print "gold_train: %s annotations" % len(gold_train)
print "gold_validation: %s annotations" % len(gold_validation)
print gold_train[0]

gold_train: 73 annotations
gold_validation: 25 annotations
('PHGLS20125-1', '-65')


In the end, we want a dictionary with {candidate UID : boolean}

In [103]:
gt_dict = {}
for c in candidates.get_candidates():
    filename = (corpus.get_doc(c.doc_id).file).split('.')[0]
    temp = c.get_attrib_span('words')
    label = (filename, temp) in gold_train
    gt_dict[c.uid] = label
print gt_dict.items()[0]
print "%s out of %s candidates are True" % (sum(gt_dict.values()), len(gt_dict))

('93-1-9-0:0-2', False)
47 out of 163 candidates are True


Save the results as a pickle

In [105]:
import cPickle
pkl_f = 'data/hardware_gold_train.pkl'
with open(pkl_f, 'w+') as f:
    cPickle.dump(gt_dict, f)